## 步骤 1 - 安装所需的 Python 环境及包

## 步骤 2 - 初步读入并清洗数据

In [1]:
import pandas as pd

df = pd.read_csv("../tweets.csv")
df.head(5)

,text,label,roberta,roberta_score,gpt2,gpt2_score
0,@user @user what do these '1/2 naked pics' hav...,neutral,0,0.8047260642051697,LABEL_2,0.9134505987167358
1,OH: âI had a blue penis while I was thisâ?[...,neutral,1,0.8669487237930298,LABEL_1,0.7534046173095703
2,"@user @user That's coming, but I think the vic...",neutral,1,0.7637239098548889,LABEL_2,0.9999619722366333
3,I think I may be finally in with the in crowd ...,positive,2,0.7740470767021179,LABEL_2,0.8987836837768555
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",negative,1,0.41639748215675354,LABEL_2,0.9864314198493958


我们可以发现，Roberta 使用 0/1/2 来分别代表 negative/neutral/positive，而 gpt2 使用 LABEL_0/1/2 来代表 negative/neutral/positive。为了方便后续处理，我们需要进行清理。

In [2]:
def label_map_gpt2(x):
	if x == "LABEL_0":
		return "negative"
	elif x == "LABEL_1":
		return "neutral"
	elif x == "LABEL_2":
		return "positive"
	else:		# illegal output, set to unknown
		return "unknown"

	
def label_map_roberta(x):
	if x == "0":
		return "negative"
	elif x == "1":
		return "neutral"
	elif x == "2":
		return "positive"
	else:		# illegal output, set to unknown
		return "unknown"

df['gpt2'] = df['gpt2'].map(label_map_gpt2)
df['roberta'] = df['roberta'].map(label_map_roberta)
df.head(5)

,text,label,roberta,roberta_score,gpt2,gpt2_score
0,@user @user what do these '1/2 naked pics' hav...,neutral,negative,0.8047260642051697,positive,0.9134505987167358
1,OH: âI had a blue penis while I was thisâ?[...,neutral,neutral,0.8669487237930298,neutral,0.7534046173095703
2,"@user @user That's coming, but I think the vic...",neutral,neutral,0.7637239098548889,positive,0.9999619722366333
3,I think I may be finally in with the in crowd ...,positive,positive,0.7740470767021179,positive,0.8987836837768555
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",negative,neutral,0.41639748215675354,positive,0.9864314198493958


Zeno 处理要求我们增加 input_length 列与 id 列，对数据进行进一步处理

In [3]:
df["input_length"] = df["text"].str.len()
df['id'] = df.index
df.head(5)

,text,label,roberta,roberta_score,gpt2,gpt2_score,input_length,id
0,@user @user what do these '1/2 naked pics' hav...,neutral,negative,0.8047260642051697,positive,0.9134505987167358,96,0
1,OH: âI had a blue penis while I was thisâ?[...,neutral,neutral,0.8669487237930298,neutral,0.7534046173095703,75,1
2,"@user @user That's coming, but I think the vic...",neutral,neutral,0.7637239098548889,positive,0.9999619722366333,87,2
3,I think I may be finally in with the in crowd ...,positive,positive,0.7740470767021179,positive,0.8987836837768555,83,3
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",negative,neutral,0.41639748215675354,positive,0.9864314198493958,133,4


## 步骤 3 - 启动 Zeno 进行模型分析

创建一个 [Zeno](https://hub.zenoml.com/account) 账号，阅读如下代码并正确运行，运行完成后你将在个人账户下看到创建的 projects

In [4]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd

# load API key from .env file
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="./.env")
ZENO_API_KEY = os.getenv("ZENO_API_KEY")

client = ZenoClient(ZENO_API_KEY)

# 创建项目
proj = client.create_project(
	name="Twitter Sentiment Analysis",
	view="text-classification",
	metrics=[
		ZenoMetric(name="roberta_accuracy", type="mean", columns=["roberta_correct"]),
		ZenoMetric(name="gpt2_accuracy", type="mean", columns=["gpt2_correct"]),
	]
)

df["id"] = df["id"].astype(str)
proj.upload_dataset(df, id_column="id", data_column='text', label_column="label")

# 为 Roberta 模型分别创建系统数据框
df_roberta = pd.DataFrame({
	"id": df["id"],
	"output": df["roberta"],
	"roberta_correct": (df["roberta"] == df["label"]).astype(int)
})
df_roberta["id"] = df_roberta["id"].astype(str)
proj.upload_system(df_roberta, name="Roberta", id_column="id", output_column="output")


# 为 GPT-2 模型分别创建系统数据框
df_gpt2 = pd.DataFrame({
	"id": df["id"],
	"output": df["gpt2"],
	"gpt2_correct": (df["gpt2"] == df["label"]).astype(int)
})
df_gpt2["id"] = df_gpt2["id"].astype(str)
proj.upload_system(df_gpt2, name="GPT-2", id_column="id", output_column="output")

Successfully updated project.
Access your project at  https://hub.zenoml.com/project/c9f17869-252d-420d-a678-b65738afae58/Twitter%20Sentiment%20Analysis


  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded data


  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system


  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system


## 步骤 4 - 创建分析切片并进一步分析

你一共需要利用 Zeno 提供的接口创建 5 个不同的切片。

创建切片可以直接通过点击 "+" 按钮来进行，可以通过基本值匹配或正则表达式的方式创建，具体用法参见[文档](https://zenoml.com/docs/intro/)
有关 Zeno 中更多有趣的用法可以参考 Zeno 仓库中的 [README](https://github.com/zeno-ml/zeno)。




创建的五个切片如下 (在个人 Zeno 账户中创建，针对每一个创建的切片，用一两句话总结模型在其上的表现特点): 

1. `with_tag`: 有标签的推文（含有"#"）
   - Roberta: 0.63; GPT-2: 0.38
   - 两个模型在含标签推文上的表现都与整体水平相近。这可能是因为标签(#)往往简短且信息量有限, 因而模型从中获得的额外情感线索也较少.
2. `with_postive`: 有强烈正向情感的词语 (含有'love', 'happy', 'fantastic')
   - Roberta: 0.76; GPT-2: 0.69
   - 明确的正向情感词汇为模型提供了强信号，Roberta在识别正向情感上表现优秀，GPT-2也有显著提升.
3. `short_sentence` 短文本推文 (input_length < 50)
   - Roberta: 0.71; GPT-2: 0.52
   - Roberta在短文本上表现出色，超过整体准确率，说明其能有效捕捉简短文本的情感。GPT-2在短文本上也有较好表现，可能是因为输入简洁，干扰信息少. 猜测也可能是短句数据中通常情感更为直接明确，模型更易于做出正确判断.
4. `long_sentence` 长文本推文 (input_length > 100)
   - Roberta: 0.60; GPT-2: 0.25
   - 在长文本上, 两个模型的表现均低于整体水平. 可能是模型在处理包含更多上下文信息的长文本时遇到困难.
5. `neutral_sentiment` 中性情感推文 (label == "neutral")
   - Roberta: 0.61; GPT-2: 0.30
   - 中性文本往往缺乏明显情感线索, 两个模型表现均低于整体水平.	

![result](screenshot.png)

## 提交：
1. 本 notebook，保留你书写的代码与输出结果
2. 一张截图，显示你创建的 5 个切片